<a href="https://colab.research.google.com/github/Gus-1003/GeoProcessamento_Estudos_Dados/blob/main/Sidra_RN_Produ%C3%A7%C3%B5esTemporarias/Sidra_RN_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Produção de Culturas temporarias no Rio Grande do Norte - UFRN

- análise de uma culturas agrícolas (dados de 2020)

In [ ]:
!pip install geopandas shapely mapclassify libpysal splot esda pysal
!pip install --upgrade pyshp
!pip install --upgrade descartes

## Leitura de Pacotes e bibliotecas


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as sm

# para gráficos
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


import shapefile

# para a análise de dados espaciais
import geopandas as gp
import pysal as ps
import splot
import mapclassify as mc
from libpysal.weights import Queen
from libpysal import weights
from esda import Moran, Moran_Local, G_Local
from splot.esda import plot_moran, moran_scatterplot, lisa_cluster, plot_local_autocorrelation

## Leitura de arquivos no Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/

geojson_data = shapefile.Reader(shp="/content/drive/MyDrive/RN_Municipios_2021.shp", dbf="https://drive.google.com/file/d/1FMx-Tvq3VBrO2fGP4fwOEwrKaiAN2yG-/view?usp=sharing")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


In [ ]:
type(geojson_data)

shapefile.Reader

In [ ]:
gp.GeoDataFrame.from_features(shp="/content/drive/MyDrive/RN_Municipios_2021.shp")

AttributeError: ignored

In [ ]:
# ler dados de interesse
dados = pd.read_csv('https://raw.githubusercontent.com/Gus-1003/GeoProcessamento_Estudos_Dados/main/Sidra_RN_Produ%C3%A7%C3%B5esTemporarias/Base_Principal.csv', delimiter=";", quotechar='"')

In [ ]:
# ler dados de interesse
geodf = gp.read_file(geojson_data)   # ler shapefile

# mesclar shapefile com dataframe usando a coluna mun
geodf.rename(columns={'Cód:': 'mun'}, inplace=True)  # mudar nome da coluna com cód. para 'mun'
geodf['mun'] = geodf.mun.astype(int)  # transformar códigos em inteiros
br = pd.merge(geodf, dados, on='mun', suffixes=('', '_y'))  # mesclar o dataframe e o shapefile em um objeto
br = br.loc[:, br.columns != 'NM_MUNICIP']  # tirar a coluna NM_MUNICIP

AttributeError: ignored

In [ ]:
br.head()

In [ ]:
br.columns

In [ ]:
variavel = 'milho'      # escolher a variável e colocar entre aspas

In [ ]:
# resumo estatístico
br[variavel].describe()

In [ ]:
# histograma
br[variavel].hist();

In [ ]:
# boxplot
sns.boxplot(br[variavel], orient='v');

In [ ]:
# visualizar os municípios em ordem decrescente (ou crescente)
br.sort_values(variavel, ascending=False)[['nome_mun', 'nome_uf', 'nome_meso', variavel]].head(10)

In [ ]:
# mapa temático - simples
br.plot(variavel, legend=True);

In [ ]:
# mapa temático - quantiles
br.plot(column=variavel, figsize=(10, 10), scheme='Quantiles', cmap='OrRd', legend=True, k=4);

In [ ]:
# mapa temático - equal_interval
br.plot(column=variavel, figsize=(10, 10), scheme='Equal_Interval', cmap='OrRd', legend=True, k=4);

In [ ]:
# intervalos de valores definidos pelo usuário
referencia = [10000, 500000, 1000000, 4000000]     # valores de referência considera 'abaixo que'
labels = ['a: 0 - 10 mil', 'b: 10 mil - 500 mil', 'c: 500 mil - 1 milhão', 'd: 1 milhão - 4 milhões']  # letras por causa de ordenação alfabética
valores = br[variavel]
quantis = mc.UserDefined(valores, referencia)
cl = [labels[i] for i in quantis.yb]
df = br.assign(cl=cl)
df.cl = pd.Categorical(df.cl, ordered=True, categories=labels)
df.plot(column='cl', figsize=(10, 10), cmap='OrRd', legend=True);

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(br)
w.transform = 'r'
# calcular I de Moran global para a variável escolhida
y = br[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
br.shape

In [ ]:
# ilhas
br.query('index == 1523 or index == 3493')

In [ ]:
# retirar ilhas
br = br.drop([1523, 3493]) # retira F. Noronha e Ilhabela

O sistema de referência WGS84 (*World Geodetic System*, 1984) é bastante utilizado em sistemas GPS e aplicativos da Google, por exemplo). A Google utiliza informações espaciais que são armazenadas como WGS84 e projetadas para WGS84 (EPSG: 4326) para o Google Earth e Pseudo Mercator (EPSG: 3857) para o Google Maps.

Fonte: https://geopandas.org/en/stable/docs/user_guide/projections.html



In [ ]:
# ajustando o sistema de referência de coordenadas
br = br.to_crs(epsg=3395)
br.shape

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(br)
w.transform = 'r'
# calcular I de Moran global para a variável escolhida
y = br[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
# valor-p   nível de significância: 0,05
moran.p_sim

In [ ]:
# só o diagrama de dispersão
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);

In [ ]:
# mapa LISA
lisa_cluster(moran_loc, br, p=0.05, figsize = (9,9));

# Escolher partes do Brasil:

Unir estados para formar as regiões do Brasil.

In [ ]:
br.nome_uf.unique()

In [ ]:
SE = br.query('nome_uf == ["Minas Gerais", "São Paulo", "Rio de Janeiro", "Espírito Santo"]')

In [ ]:
SE.shape

In [ ]:
SE.plot();

In [ ]:
# mapa temático - quantiles
SE.plot(column=variavel, figsize=(10, 10), scheme='Quantiles', cmap='OrRd', legend=True, k=4);

In [ ]:
# resumo estatístico - para auxiliar na construção dos intervalos
SE[variavel].describe()

In [ ]:
# intervalos de valores definidos pelo usuário
referencia = [100, 7000, 3480000]     # valores de referência considera 'abaixo que'
labels = ['a: 0 - 100', 'b: 100 - 7 mil', 'c: 7 mil - 348 mil']  # letras por causa de ordenação alfabética
valores = SE[variavel]
quantis = mc.UserDefined(valores, referencia)
cl = [labels[i] for i in quantis.yb]
df = SE.assign(cl=cl)
df.cl = pd.Categorical(df.cl, ordered=True, categories=labels)
df.plot(column='cl', figsize=(10, 10), cmap='OrRd', legend=True);

# Tarefa 1:

Eu escolhi unir os estados da região Sudeste. Para a mesma variável utilizada por mim:

- escolha estados de outra região do Brasil
- obter o mapa temático por quantis
- obter o mapa temático definido pelo usuário, da forma que for mais adequada para a região escolhida por você (escolha quantos intervalos quiser e faça os ajustes necessários nos valores)

# Diferentes níveis de agregação:

Dividir em microrregiões do Brasil

In [ ]:
# usando a função buffer
br['geometry'] = br.geometry.buffer(0)

In [ ]:
# agrupar municípios em microrregiões
# somando os valores da variável para os municípios contidos em cada microrregião
micros = br.dissolve(by=['micro', 'nome_micro', 'nome_uf'], aggfunc='sum')
micros.head(3)

In [ ]:
micros.shape

In [ ]:
# reiniciar índices e retirar as colunas que não serão usadas
micros = micros.reset_index()
retirar = ['mun', 'uf', 'meso', 'cod_rgi', 'cod_rgint', 'codmun6', 'regiao', 'area']
micros = micros.drop(retirar, axis=1)
micros.head(3)

In [ ]:
micros = micros.to_crs(epsg=3395)

In [ ]:
# resumo estatístico
micros[variavel].describe()

In [ ]:
micros[variavel].hist();

In [ ]:
# boxplot
sns.boxplot(micros[variavel], orient='v');

In [ ]:
# visualizar as microrregiões em ordem decrescente (ou crescente)
micros.sort_values(variavel, ascending=False)[['nome_micro', 'nome_uf', variavel]].head(10)

In [ ]:
# mapa simples
micros.plot();

In [ ]:
# mapa temático - quantiles
micros.plot(column=variavel, figsize=(10, 10), scheme='Quantiles', cmap='OrRd', legend=True, k=4);

In [ ]:
# mapa temático - equal_interval
micros.plot(column=variavel, figsize=(10, 10), scheme='Equal_Interval', cmap='OrRd', legend=True, k=4);

In [ ]:
# mapa temático - fisher_jenks
micros.plot(column=variavel, figsize=(10, 10), scheme='fisher_jenks', cmap='OrRd', legend=True, k=4);

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(micros)
w.transform = 'r'
# calcular I de Moran global para a variável escolhida
y = micros[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
# valor-p
moran.p_sim

In [ ]:
# mapa LISA
moran_loc = Moran_Local(y, w)
lisa_cluster(moran_loc, micros, p=0.05, figsize = (9,9));

In [ ]:
# mapa LISA com nomes dos significativos
fig, ax = plt.subplots(1, figsize=(20, 20))
lisa_cluster(moran_loc, micros, p=0.05, ax=ax, figsize = (9,9));
sig = moran_loc.p_sim < 0.05  
posicoes = np.where(sig)
escolhidos = micros.iloc[posicoes[0]]
for j in escolhidos.index:
    ax.text(escolhidos.geometry.centroid[j].coords[0][0], escolhidos.geometry.centroid[j].coords[0][1], 
         escolhidos.nome_micro[j],
         fontsize=10, horizontalalignment='center', verticalalignment='bottom')

# Tarefa 2

Eu utilizei o nível de agregação microrregiões para os dados do Brasil. Usando a mesma variável já utilizada, escolha outro nível de agregação (mesorregiões, regiões imediatas, regiões intermediárias ou mesmo estados) e faça a análise completa:

- obtenha o mapa temático que você achar mais adequado (quantis, intervalos iguais, fisher-jenks ou definido por você) e ANALISE (apontando como estão distribuídos os valores no território brasileiro)
- obtenha o valor do *I* de Moran e valor-*p* e ANALISE (o valor do *I* é significativo? O que isso quer dizer? Qual a interpretação prática para a variável em questão?) 
- obtenha o mapa LISA com os nomes das observações significativas e ANALISE (apontando quais foram os *clusters* espaciais, *outliers* espaciais)



# Tarefa 3

Compare suas conclusões obtidas na tarefa 2 (com o nível de agregação escolhido por você) com o que foi obtido por mim para as microrregiões:

- há semelhanças?
- há diferenças?
- aponte o que notou de interessante ao comparar os dois níveis de agragação (microrregiões *versus* nível escolhido por você)

Digite aqui sua análise da tarefa 3.


